# Module 5 - Linear Regression

During the data project life cycle, building a predictive model may be the next step in the process, if a goal of the project is to generate information about possible future outcomes. Although the EDA and data cleaning/preparation phases are a significant portion of the time spent for the project, choosing, building, and evaluating a predictive model can be a time consuming process as well. 

In this lesson, as well as Modules 6 & 7, we will use the Scikit-learn library (`sklearn`) to build machine learning algorithms and produce predictive results. This lesson will focus on **linear regression**, a function used to predict continuous numerical outcomes, to build a model to predict student grades.

***Note:*** We've explored many techniques and concepts for analyzing and preparing the `gradedata.csv` dataset. For simplicity, we will use the "clean" version of the dataset. Just know that with a "real" dataset, you would have to go through all the previous steps that we've covered in Modules 2-4 to get to this stage.

In [ ]:
# import libraries
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
filepath = "datasets/gradedata.csv"

df = pd.read_csv(filepath)
df.head()

## Correlation

**Correlation** is the linear (straight line) relationship between one feature (column) to another. Features can be **positively correlated**, meaning that they both move in the same direction (if one increases, so does the other and if one decreases, so does the other) or **negatively correlated**, meaning that they move in opposite directions (if one increases, the other decreases). Correlation values are on a scale from `-1` to `+1`. Features that are positively correlated are closer to `+1` and features that are negatively correlated are closer to `-1`. All features are perfectly positively correlated (exact value of 1) with itself.

The `.corr()` function will calculate the correlation values of all numeric column types and display them in a dataframe. In this example, because `grade` will be the **target** (thing to predict) we want to identify which **features** (other columns in the dataset) have a strong correlation to `grade`. We may also want to look for correlation among other features as well, since they may create misleading signals in the model, due to their influence on each other.

In [ ]:
# correlation values for age, exercise, hours, and grade
df.corr()

## Linear Regression

Linear regression is used to predict a continuous numerical value(s) for a **target** variable (the column that is being predicted). With one column as a predictor (attribute/feature), a linear regression model is mathematically represented by the formula:
### \begin{align}  y = mx + b \end{align}
Where ***y*** is the target variable, ***x*** is the predictor, ***m*** is the slope (weight of ***x***), and ***b*** is the y-intercept, which is the starting value of ***y*** when m*x=0. Below is a linear regression line graphed to predict student grades based on the number of hours studied for an exam.

<center><img src='https://s3.amazonaws.com/stackabuse/media/linear-regression-python-scikit-learn-1.png'></center>

Source: [Stack Abuse](https://s3.amazonaws.com/stackabuse/media/linear-regression-python-scikit-learn-1.png)

## Predictive Modeling

The correlation table identified `hours` as having the strongest (positive) correlation, so we will first build our linear regression with this feature only to predict student grades. Then, we will add other features such as `age` and `exercise` to see if the model improves.

There are a few final tasks that need to be done before the data is given to the model:

- Create a dataframe with only the attributes (features used to predict) and target (outcome to predict) columns that will be used in the model
- Separate the attributes from the target 
- Shuffle the order of the rows in the dataset, then separate into a dataset for training (for the model to learn from) and testing (to see how well it predicts with new data)

### Choose attributes and target

In [ ]:
#dataframe with 'hours' (attribute) and 'grade' (target)
modeldf = df[['hours', 'grade']]

In [ ]:
# display the dataframe being used for the model
modeldf.head()

### Separate attributes and target

The target is the column of data we are teaching the model to predict. In math, this information is typically represented as the variable `y`, so we keep the same conventions. Attributes (characteristics) that calculate/predict `y` are stored into a variable called `X`. Although `y` is a single column, `X` is a dataframe of all the attribute columns.

In [ ]:
# attributes are all the columns EXCEPT 'grade' (target)
X = modeldf.drop('grade', axis=1)

In [ ]:
# 'grade' is target ('y' variable) 
y = modeldf['grade']

### Separate training and test data

Scikit-learn's `train_test_split()` function takes the attribute columns (`X` variable) and target column (`y` variable), then shuffles the rows using the `random_state=` argument, which will trigger a randomizing sequence (see the [`random_state` documentation](https://scikit-learn.org/stable/glossary.html#term-random-state) for more information). By default, `test_size=` will separate 25% of the dataset as the test set, leaving the other 75% for the training set. However, you can adjust the value of the ratio split.

`train_test_split()` then generates four outputs in this order - a dataframe of the attributes for the training set (`X_train`), a dataframe of the attributes for the test set (`X_test`), a column of the target for the training set (`y_train`), and a column of the target for the test set (`y_test`).

In [ ]:
# separate into training data (75%) and test data (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Train the model

The `LinearRegression()` function, in combination with the `.fit()` function, will take the `X_train` and `y_train` dataset, and calculate the attributes' parameters (weights) to produce a linear trend line that best fits the data.

When the model finishes "learning" with the training data, we will evaluate its performance. the `.score()` function when used with linear regression, calculates an **R-squared** value as the measure of the model's fit to the training data.

In [ ]:
# assign linear regression function to a variable
model = LinearRegression()

# model = LinearRegression(fit_intercept=False)

In [ ]:
# give training data to learn
model.fit(X_train, y_train)

In [ ]:
# calculate R-squared (how well trend line fits data)
model.score(X_train, y_train)

### Evaluate model with test data

In [ ]:
# R-squared value for test data
model.score(X_test, y_test)

In [ ]:
# generate predictive values from test data
y_pred = model.predict(X_test)

In [ ]:
# compare actual student grade to grade predicted by model (first 5 rows of test data)
print("Actual student grades (first 5): ", list(y_test[:5]))
print("Predicted student grades (first 5): ", y_pred[:5])

### Observations:

Using the R-squared value as a measurement of performance, this model doesn't seem to predict very well. However, unlike a statistical analysis where improving the R-squared value to be close to `1.0` is highly desired, in a predictive model, we want to leave a bit of error for flexibility when predicting future output.

Some things to think about for improvement:

- During the earlier analysis (Modules 2-4), we found that students had grades within a range of values, given some attribute (such as hours of study). Most of the students seem to have done well on the exam, and there's not a lot of variation in the data. If it were possible to collect additional features about the students (such as commuting time), that might be a more significant factor to predicting student grades.
- Linear regression might not be the best machine learning algorithm to use on this dataset, in addition to the actual values that we want to predict (continuous numbers). If we aggregated students by letter grade instead and built a model to predict which letter grade a student is likely to recieve, that might be a better method of forecasting future student performance on the exam.